In [ ]:
# import moviepy.editor as mp
# from pydub import AudioSegment
# from pydub.silence import split_on_silence
# from vosk import Model, KaldiRecognizer, SetLogLevel
# import numpy as np

# # Set log level to suppress Vosk log messages
# SetLogLevel(0)

# # Step 1: Extract audio from video
# video = mp.VideoFileClip("um.mp4")
# audio = video.audio

# # Step 2: Convert audio to WAV for speech recognition
# audio.export("audio.wav", format="wav", codec="pcm_s16le")

# # Step 3: Speech recognition using Vosk
# model_path = "../vosk-model-en-us-0.22"
# try:
#     model = Model(model_path)
#     print("Model loaded successfully.")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     # Handle the error or exit the program

# # Step 4: Process audio chunks
# audio_data = AudioSegment.from_wav("audio.wav")
# chunks = split_on_silence(audio_data, silence_thresh=-40)
# recognized_text = ""

# for chunk in chunks:
#     # Convert audio data to numpy array of 16-bit signed integers
#     audio_np = np.array(chunk.get_array_of_samples(), dtype=np.int16)
#     recognizer = KaldiRecognizer(model, audio_np)
#     recognized_text += recognizer.FinalResult()

# # Step 5: Identify "um" occurrences and cut corresponding video segments
# um_segments = [(start, end) for start, end in zip(chunks[:-1], chunks[1:]) if "um" in recognized_text[start:end]]

# edited_segments = []
# for start, end in um_segments:
#     edited_segments.append(video.subclip(start / 1000, end / 1000))  # Convert milliseconds to seconds

# final_video = mp.concatenate_videoclips(edited_segments)

# # Step 6: Export final video
# final_video.write_videofile("umRemove.mp4", codec="libx264", audio_codec="aac")


In [1]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import split_on_silence
import wit
import requests
import os
import csv

# Get the current working directory
current_dir = os.getcwd()

# Move one directory up from the current directory
parent_dir = os.path.dirname(current_dir)

# Construct the path to witToken.txt
wit_path = os.path.join(parent_dir, 'witToken.csv')

# Read the access token from witToken.txt
with open(wit_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if row:
            access_token = row[0]

# Set up Wit.ai
wit.access_token = access_token

# Step 1: Extract audio from video
video = mp.VideoFileClip("um.mp4")
audio = video.audio

# Step 2: Convert audio to WAV for speech recognition
audio.write_audiofile("audio.wav", codec="pcm_s16le")

# Step 3: Process audio chunks
audio_data = AudioSegment.from_wav("audio.wav")
# Adjust parameters for split_on_silence
# Adjust parameters for split_on_silence
# chunks = split_on_silence(
#     audio_data,
#     silence_thresh=-40,
#     min_silence_len=500,    # Reduce the minimum silence length
#     silence_chunk_len=500,  # Length of each chunk (if a chunk has no speech, it will be extended up to this length)
#     keep_silence=100,       # Keep this amount of silence at the beginning and end of each chunk
# )

recognized_text = ""

# Directory to save individual audio chunks
output_directory = "audio_chunks"
os.makedirs(output_directory, exist_ok=True)

# Initialize a list to store start and end times of each chunk
chunk_times = []

# Process audio chunks
for i, chunk in enumerate(chunks):
    # Print the duration of each chunk
     # Export the individual chunk to a separate audio file
    chunk.export(os.path.join(output_directory, f"chunk_{i + 1}.wav"), format="wav")
    print(f"Chunk {i + 1} duration: {chunk.duration_seconds} seconds")

    # Convert audio data to numpy array of 16-bit signed integers
    audio_np = chunk.get_array_of_samples()

    # Upload the audio to Wit.ai for recognition
    wit_url = "https://api.wit.ai/speech"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "audio/wav",
    }
    response = requests.post(wit_url, headers=headers, data=chunk.raw_data)

    # Print the response from Wit.ai API
    print(f"Response from Wit.ai API: {response.json()}")

    # Extract recognized text from the response
    recognized_text += response.json().get("_text", "")

    # Store the start and end times of the chunk
    start_time = sum(chunk_times[-1]) if chunk_times else 0  # Calculate cumulative duration
    end_time = start_time + chunk.duration_seconds
    chunk_times.append((start_time, end_time))

# Save the complete transcript to a text file
with open("transcript.txt", "w") as transcript_file:
    transcript_file.write(recognized_text)

 
    



MoviePy - Writing audio in audio.wav


MoviePy - Done.


NameError: name 'chunks' is not defined

In [ ]:
# # `pip3 install assemblyai` (macOS)
# # `pip install assemblyai` (Windows)

# import assemblyai as aai

# aai.settings.api_key = "804b4c0e83a940f5aaf099c7486000ad"
# transcriber = aai.Transcriber()

# transcript = transcriber.transcribe("um.mp4")
# # transcript = transcriber.transcribe("./my-local-audio-file.wav")

# print(transcript.text)

In [ ]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import split_on_silence
import wit
import requests
import os
import csv
import speech_recognition as sr


# Get the current working directory
current_dir = os.getcwd()

# Move one directory up from the current directory
parent_dir = os.path.dirname(current_dir)

# Construct the path to witToken.txt
wit_path = os.path.join(parent_dir, 'witToken.csv')

# Read the access token from witToken.txt
with open(wit_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if row:
            access_token = row[0]

# Set up Wit.ai
wit.access_token = access_token

# Step 1: Extract audio from video
video = mp.VideoFileClip("um.mp4")
audio = video.audio

# Step 2: Convert audio to WAV for speech recognition
audio.write_audiofile("audio.wav", codec="pcm_s16le")

# Step 3: Process audio chunks
audio_data = AudioSegment.from_wav("audio.wav")
# Replace split_on_silence with manual processing
chunks = audio_data.split_to_mono()

recognized_text = ""

# Directory to save individual audio chunks
output_directory = "audio_chunks"
os.makedirs(output_directory, exist_ok=True)

# Initialize a list to store start and end times of each chunk
chunk_times = []

# Word to be removed
word_to_remove = b"ummm"

# Process audio chunks
for i, chunk in enumerate(chunks):
    # Export the individual chunk to a separate audio file
    chunk.export(os.path.join(output_directory, f"chunk_{i + 1}.wav"), format="wav")
    print(f"Chunk {i + 1} duration: {chunk.duration_seconds} seconds")

    # Convert audio data to numpy array of 16-bit signed integers
    audio_np = chunk.get_array_of_samples()

    # Use local speech recognition
    recognizer = sr.Recognizer()
    audio_chunk = sr.AudioData(audio_np, chunk.frame_rate, 2)
    try:
        recognized_text_chunk = recognizer.recognize_google(audio_chunk)
        print(f"Recognized text from local recognition: {recognized_text_chunk}")
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
        recognized_text_chunk = ""

    # Remove the specified word from the recognized text
    recognized_text_chunk = recognized_text_chunk.replace(word_to_remove.decode(), "")

    # Append the modified chunk text to the complete recognized text
    recognized_text += recognized_text_chunk
    
    # Save the recognized text for each chunk to a separate text file
    with open(os.path.join(output_directory, f"transcript_chunk_{i + 1}.txt"), "w") as transcript_file:
        transcript_file.write(recognized_text_chunk)

    # Store the start and end times of the chunk
    start_time = sum(chunk_times[-1]) if chunk_times else 0  # Calculate cumulative duration
    end_time = start_time + chunk.duration_seconds
    chunk_times.append((start_time, end_time))




In [ ]:
# from ibm_watson import SpeechToTextV1
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# authenticator = IAMAuthenticator("YOUR_API_KEY")
# speech_to_text = SpeechToTextV1(authenticator=authenticator)

# audio_file = "path/to/audio/file.wav"

# with open(audio_file, "rb") as audio_file:
#     result = speech_to_text.recognize(audio=audio_file, content_type="audio/wav")
#     print("Transcript: {}".format(result.result["results"][0]["alternatives"][0]["transcript"]))


In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from pydub import AudioSegment
import torchaudio
import torch


# Load model directly
processor = AutoProcessor.from_pretrained("Baghdad99/saad-speech-recognition-hausa-audio-to-text")
model = AutoModelForSpeechSeq2Seq.from_pretrained("Baghdad99/saad-speech-recognition-hausa-audio-to-text")

# Step 1: Extract audio from video using pydub
video_path = "um.mp4"
output_audio_path = "audio.wav"
output_text_path = "transcript.txt"
output_video_path = "processed_video.mp4"

# Load video with pydub
audio = AudioSegment.from_file(video_path)

# Export audio to desired format (16kHz mono)
audio.export(output_audio_path, format="wav")



waveform, sample_rate = torchaudio.load(output_audio_path)

# Step 3: Preprocess the audio file for the model
input_values = processor(waveform.numpy(), return_tensors="pt", padding="longest").input_values

# Step 4: Perform inference using the loaded model
with torch.no_grad():
    logits = model(input_values).logits

# Step 5: Decode the logits to text
transcription = processor.batch_decode(logits, skip_special_tokens=False)[0]

# Step 6: Save the transcription to a text file
with open(output_text_path, "w") as f:
    f.write(transcription)

# Optionally, you can also save the processed audio if needed
processed_waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
torchaudio.save("processed_audio.wav", processed_waveform, sample_rate)





Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
